In [181]:
#Steps followed
#Parsed the xml file
#Found the ip addresses of the users which accessed the service via the Internet.org proxy by checking the field of proto
#Found countries to which the ip addresses belong
#Found the frequencies of each country and created the data.csv file
import xmltodict
import sys
import requests
from tqdm import tqdm
import csv

In [182]:
def open_file(filename):
    with open(filename,"r") as xml:
        xml_dict = xmltodict.parse(xml.read())
        xml_.close()
    return xml_dict

In [183]:
filename = input("Enter filename: ")           #give name of the xml file, for example "http.xml"

Enter filename: http.xml


In [184]:
xml_dict = open_file(filename)       #opening the xml file
print("File opened")

File opened


In [185]:
def find_ips(xml_dict):
    ips = {}
    for packet in xml_dict['pdml']['packet']:      #iterating on packets
        protos = packet['proto']
        proxy = False                      
        freebasics = False                 
        for proto in protos:
            fields = proto['field']
            for field in fields:
                if field['@show'] == 'Via: Internet.org  ':   #flag to check if request if for freebasics
                    freebasics = True
                if field['@name'] == 'http.x_forwarded_for':  #flag for checking whether the request is through proxy
                    proxy = True
                    ip_id = field['@show']
        if proxy and freebasics :     # count the ip only if both flags satisfied
            ips[ip_id] = 1;
    print("Total ips extracted which uses proxy and freebasics: ",len(ips))
    return ips

In [186]:
ips = find_ips(xml_dict)        #finding the required ips from the xml 

Total ips extracted which uses proxy and freebasics:  481


In [187]:
# ips    #list of all the ips

In [188]:
def find_countries(ips):
    countries = [];
    for i in tqdm(ips):
        country = requests.get("https://geolocation-db.com/json/"+i+"&position=true").json()["country_name"]
        countries.append(country)
    return countries

In [189]:
countries = find_countries(ips)     #finding countries for the extracted ips

100%|██████████| 481/481 [05:16<00:00,  1.52it/s]


In [190]:
# countries #list of countries

In [191]:
def CountFrequency(countries):   
    freq = {} 
    for item in countries: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq

In [192]:
freq = CountFrequency(countries)    #creating frequency list for the countries

In [193]:
with open('data.csv', 'w') as f:        #creating the .csv file
    w = csv.DictWriter(f, freq.keys())
    w.writeheader()
    w.writerow(freq)